In [1]:
# Importing dependencies
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

from sklearn.model_selection import StratifiedGroupKFold,train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve, learning_curve

In [2]:
# loading the dataset
df = pd.read_pickle('./pickled/Train_set_scaled')